## Buat Spark Session dan Load Data dari HDFS

In [14]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType, LongType

# Inisialisasi SparkSession
spark = SparkSession.builder \
    .appName("ReadFromHDFS_ExplicitPath") \
    .getOrCreate()

# Path HDFS yang lebih eksplisit
path_hdfs = "hdfs://localhost:9000/raw_data/data.csv"

# Nama kolom (jika diperlukan, misalnya jika tidak ada header atau untuk skema manual)
nama_kolom = ["tahun", "kabupaten_kota", "wpp", "volume_produksi_ton", "nilai_produksi_rp_000", "provinsi", "kelompok", "jenis_ikan"]

try:
    df_hdfs = spark.read.format("csv") \
        .option("header", "true") \
        .option("delimiter", ",") \
        .option("inferSchema", "true") \
        .load(path_hdfs)

    print(f"Berhasil membaca dari HDFS: {path_hdfs}")
    df_hdfs.printSchema()
    df_hdfs.show(5, truncate=False)
    print(f"Jumlah baris yang dibaca dari HDFS: {df_hdfs.count()}")

except Exception as e:
    print(f"Error membaca file dari HDFS {path_hdfs}: {e}")
    print("Pastikan URI HDFS benar dan NameNode berjalan.")
    print("Pastikan file ada di path tersebut dan Spark memiliki izin akses.")
    print("Periksa format file dan opsi yang digunakan.")

finally:
    # Hentikan SparkSession
    print("Done")

Berhasil membaca dari HDFS: hdfs://localhost:9000/raw_data/data.csv
root
 |-- Tahun: integer (nullable = true)
 |-- Kabupaten/Kota: string (nullable = true)
 |-- WPP: string (nullable = true)
 |-- Volume Produksi (Ton): double (nullable = true)
 |-- Nilai Produksi (Rp 000): integer (nullable = true)
 |-- Provinsi: string (nullable = true)
 |-- Kelompok: string (nullable = true)
 |-- Jenis Ikan;: string (nullable = true)

+-----+--------------+----------+---------------------+-----------------------+--------------+-----------+-----------------------------+
|Tahun|Kabupaten/Kota|WPP       |Volume Produksi (Ton)|Nilai Produksi (Rp 000)|Provinsi      |Kelompok   |Jenis Ikan;                  |
+-----+--------------+----------+---------------------+-----------------------+--------------+-----------+-----------------------------+
|2023 |AGAM          |WPP-RI-572|184.17               |3683400                |SUMATERA BARAT|ALU-ALU    |ALU-ALU BESAR; BARAKUDA BESAR|
|2023 |AGAM          |WPP-R

## Eksplorasi Data

In [15]:
from pyspark.sql.functions import col, countDistinct, avg, min, max, round

# Menampilkan jumlah total baris
print(f"Jumlah total baris: {df_hdfs.count()}")

# Menampilkan jumlah nilai unik dalam setiap kolom
df_hdfs.select([countDistinct(col(c)).alias(f"Unique_{c}") for c in df_hdfs.columns]).show()

# Menampilkan statistik deskriptif untuk kolom numerik
df_hdfs.describe().show()

# Menampilkan nilai minimum dan maksimum untuk beberapa kolom numerik
df_hdfs.select(
    min(col("Volume Produksi (Ton)")).alias("Min Volume"),
    max(col("Volume Produksi (Ton)")).alias("Max Volume"),
    min(col("Nilai Produksi (Rp 000)")).alias("Min Nilai"),
    max(col("Nilai Produksi (Rp 000)")).alias("Max Nilai")
).show()

# Menghitung rata-rata volume produksi
df_hdfs.select(round(avg(col("Volume Produksi (Ton)")), 2).alias("Rata-rata Volume Produksi")).show()

# Menampilkan jumlah produksi per provinsi
df_hdfs.groupBy("Provinsi").agg(
    round(avg(col("Volume Produksi (Ton)")), 2).alias("Rata-rata Volume"),
    round(avg(col("Nilai Produksi (Rp 000)")), 2).alias("Rata-rata Nilai"),
    countDistinct("Jenis Ikan").alias("Jumlah Jenis Ikan")
).orderBy(col("Rata-rata Volume").desc()).show()

# Menampilkan distribusi data untuk beberapa kategori
df_hdfs.groupBy("Kelompok").count().orderBy(col("count").desc()).show()
df_hdfs.groupBy("Jenis Ikan").count().orderBy(col("count").desc()).show()

Jumlah total baris: 8144


+------------+---------------------+----------+----------------------------+------------------------------+---------------+---------------+------------------+
|Unique_Tahun|Unique_Kabupaten/Kota|Unique_WPP|Unique_Volume Produksi (Ton)|Unique_Nilai Produksi (Rp 000)|Unique_Provinsi|Unique_Kelompok|Unique_Jenis Ikan;|
+------------+---------------------+----------+----------------------------+------------------------------+---------------+---------------+------------------+
|           5|                   45|         7|                        6918|                          7835|              6|             98|               344|
+------------+---------------------+----------+----------------------------+------------------------------+---------------+---------------+------------------+



25/05/26 10:04:35 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
                                                                                

+-------+------------------+--------------+--------------------+---------------------+-----------------------+--------------+--------+------------+
|summary|             Tahun|Kabupaten/Kota|                 WPP|Volume Produksi (Ton)|Nilai Produksi (Rp 000)|      Provinsi|Kelompok| Jenis Ikan;|
+-------+------------------+--------------+--------------------+---------------------+-----------------------+--------------+--------+------------+
|  count|              8144|          8144|                8144|                 8144|                   8144|          8144|    8144|        8144|
|   mean|2020.9964390962673|          NULL|                NULL|    604.7076743614933|   1.8392105080427308E7|          NULL|    NULL|        NULL|
| stddev| 1.424893020401632|          NULL|                NULL|    1656.932747495079|    4.427455021285158E7|          NULL|    NULL|        NULL|
|    min|              2019|          AGAM|Laut Lepas Samude...|                  0.0|                     15|  

AnalysisException: [UNRESOLVED_COLUMN.WITH_SUGGESTION] A column or function parameter with name `Jenis Ikan` cannot be resolved. Did you mean one of the following? [`Jenis Ikan;`, `Kelompok`, `Tahun`, `Provinsi`, `WPP`].;
'Aggregate [Provinsi#123], [Provinsi#123, round(avg(Volume Produksi (Ton)#121), 2) AS Rata-rata Volume#966, round(avg(Nilai Produksi (Rp 000)#122), 2) AS Rata-rata Nilai#968, 'count(distinct 'Jenis Ikan) AS Jumlah Jenis Ikan#969]
+- Relation [Tahun#118,Kabupaten/Kota#119,WPP#120,Volume Produksi (Ton)#121,Nilai Produksi (Rp 000)#122,Provinsi#123,Kelompok#124,Jenis Ikan;#125] csv


## Preprocessing Data

In [16]:
df_hdfs.show()

+-----+--------------+----------+---------------------+-----------------------+--------------+-----------+--------------------+
|Tahun|Kabupaten/Kota|       WPP|Volume Produksi (Ton)|Nilai Produksi (Rp 000)|      Provinsi|   Kelompok|         Jenis Ikan;|
+-----+--------------+----------+---------------------+-----------------------+--------------+-----------+--------------------+
| 2023|          AGAM|WPP-RI-572|               184.17|                3683400|SUMATERA BARAT|    ALU-ALU|ALU-ALU BESAR; BA...|
| 2023|          AGAM|WPP-RI-572|               364.68|               12763660|SUMATERA BARAT|      BAWAL|        BAWAL HITAM;|
| 2023|          AGAM|WPP-RI-572|                16.99|                 509640|SUMATERA BARAT|    BELANAK|            BELANAK;|
| 2023|          AGAM|WPP-RI-572|               134.91|                2698280|SUMATERA BARAT|BIJI NANGKA| BIJI NANGKA KARANG;|
| 2023|          AGAM|WPP-RI-572|               457.54|                8235702|SUMATERA BARAT|BIJI NANGK